In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math

/home/team/zhaohongwei/anaconda3/envs/torch_hwzhao/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
torch

In [3]:
out = [1,2,3,4]
print(out[-1])

4


In [83]:
weight = torch.randn(3)
print(weight)
x = torch.ones(3,1,2)
ans = torch.mul(x.permute(1,2,0),weight)
ans = ans.permute(2,0,1)
print(ans.shape)
# print(torch.mul(x,weight))

tensor([0.9348, 1.6738, 0.2401])
torch.Size([3, 1, 2])


In [62]:
class Adapter(nn.Module):
    def __init__(self,
                 config=None,
                 d_model=None,
                 bottleneck=None,
                 ):
        super().__init__()
        # 按照增量阶段生成lora
        self.expert_lora = nn.ModuleDict(
                {
                    'down_proj': (nn.Linear(6, 5)),
                    'up_proj': (nn.Linear(5, 6))
                }
            )
        self.expert_loras = nn.ModuleList([self.expert_lora for i in range(3)])
        

In [67]:
net = Adapter()
print(net.expert_loras)

ModuleList(
  (0): ModuleDict(
    (down_proj): Linear(in_features=6, out_features=5, bias=True)
    (up_proj): Linear(in_features=5, out_features=6, bias=True)
  )
  (1): ModuleDict(
    (down_proj): Linear(in_features=6, out_features=5, bias=True)
    (up_proj): Linear(in_features=5, out_features=6, bias=True)
  )
  (2): ModuleDict(
    (down_proj): Linear(in_features=6, out_features=5, bias=True)
    (up_proj): Linear(in_features=5, out_features=6, bias=True)
  )
)


In [25]:
x = torch.randn(5,768)
router = nn.Linear(768,5)
out = router(x)
route_weight = nn.functional.softmax(router(x), dtype=torch.float32)
print(route_weight)
print(torch.topk(route_weight,1,largest=True))
# print(torch.unsqueeze(route_weight[:,:,0], -1))

tensor([[0.0826, 0.1179, 0.1173, 0.4351, 0.2471],
        [0.1506, 0.2258, 0.2749, 0.2319, 0.1169],
        [0.4563, 0.1899, 0.1133, 0.1241, 0.1164],
        [0.0682, 0.1515, 0.0989, 0.2225, 0.4588],
        [0.4964, 0.1529, 0.0872, 0.1904, 0.0731]], grad_fn=<SoftmaxBackward>)
torch.return_types.topk(
values=tensor([[0.4351],
        [0.2749],
        [0.4563],
        [0.4588],
        [0.4964]], grad_fn=<TopkBackward>),
indices=tensor([[3],
        [2],
        [0],
        [4],
        [0]]))


/home/team/zhaohongwei/anaconda3/envs/torch_hwzhao/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  after removing the cwd from sys.path.


In [20]:
input = torch.rand(4,5)
print(input)
print(input.topk(k=1,dim=1,largest=True))

tensor([[0.0181, 0.7638, 0.0089, 0.7393, 0.9082],
        [0.4023, 0.5285, 0.6994, 0.8291, 0.9270],
        [0.6205, 0.9296, 0.0584, 0.9661, 0.6988],
        [0.8138, 0.8475, 0.3744, 0.9592, 0.6532]])
torch.return_types.topk(
values=tensor([[0.9082],
        [0.9270],
        [0.9661],
        [0.9592]]),
indices=tensor([[4],
        [4],
        [3],
        [3]]))


In [30]:
import collections
class Adapter(nn.Module):
   def __init__(self):                        
        super(Adapter, self).__init__()
        self.expert_lora0 = (nn.Sequential(OrderedDict({
                    'down_proj': nn.init.kaiming_uniform_(nn.Linear(self.n_embd, self.down_size).weight, a=math.sqrt(5)),
                    'up_proj': nn.init.zeros_(nn.Linear(self.down_size, self.n_embd).weight)
                })))